<a href="https://colab.research.google.com/github/WarunaDissanayake1234/Movie-Review-Prediction-System/blob/main/Movie_Review_Prediction_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text as keras_text
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_lables) = imdb.load_data(num_words= VOCAB_SIZE)


In [ ]:
# lets look at one reviews
train_data[0]

In [ ]:
 # adding the padding

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [ ]:
len(train_data[54])

250

In [ ]:
# train the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation ="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop",metrics=['acc'])
history = model.fit(train_data, train_labels, epochs = 10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 32s 47ms/step - loss: 0.0526 - acc: 0.9839 - val_loss: 0.5333 - val_acc: 0.8666
Epoch 2/10
625/625 [==============================] - 14s 23ms/step - loss: 0.0466 - acc: 0.9861 - val_loss: 0.4204 - val_acc: 0.8748
Epoch 3/10
625/625 [==============================] - 11s 17ms/step - loss: 0.0388 - acc: 0.9884 - val_loss: 0.4784 - val_acc: 0.8774
Epoch 4/10
625/625 [==============================] - 9s 15ms/step - loss: 0.0300 - acc: 0.9919 - val_loss: 0.5139 - val_acc: 0.8768
Epoch 5/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0289 - acc: 0.9922 - val_loss: 0.5639 - val_acc: 0.8720
Epoch 6/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0251 - acc: 0.9930 - val_loss: 0.5564 - val_acc: 0.8646
Epoch 7/10
625/625 [==============================] - 8s 13ms/step - loss: 0.0212 - acc: 0.9940 - val_loss: 0.6042 - val_acc: 0.8718
Epoch 8/10
625/625 [==============================] - 8s 13ms/ste

In [ ]:
result = model.evaluate(test_data, test_lables)
print(result)

782/782 [==============================] - 4s 5ms/step - loss: 0.7883 - acc: 0.8467
[0.7883379459381104, 0.8466799855232239]


In [ ]:
# making prediction

# encode text to numeric
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras_text.text_to_word_sequence(text)
    # Replace words not in the word index with 0, and encode words using IMDb word index
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], maxlen=MAXLEN)

text = "That movie was jsut amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [ ]:
# Assuming you've already defined reverse_word_index and encoded as earlier in your code

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            word = reverse_word_index.get(num, '')  # Retrieve word from reverse_word_index
            if word:
                text += word + " "
    return text.rstrip()  # Remove trailing space

decoded_text = decode_integers(encoded[0])  # Assuming encoded is a list containing the sequence
print(decoded_text)



that movie was amazing so amazing


In [ ]:
# lets make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] =encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was grate"
predict(positive_review)

negative_review = "That movie sucked, I hate that movie"
predict(negative_review)

1/1 [==============================] - 0s 29ms/step
[0.7189135]
1/1 [==============================] - 0s 28ms/step
[0.08484075]
